In [ ]:
from binance.client import Client
import datetime

# Configura tu API Key y API Secret de Binance
api_key = ''
api_secret = ''

# Crea una instancia del cliente de Binance
client = Client(api_key, api_secret)

# Obtén el tiempo actual en formato Unix (milisegundos)
timestamp = client.get_server_time()['serverTime']

# Ejemplo: obtener el estado de la cuenta
account_info = client.get_account()
print(account_info)

# Ejemplo: obtener el libro de órdenes para un par de criptomonedas
depth = client.get_order_book(symbol='BTCUSDT')
print(depth)


# Convierte el timestamp a un objeto datetime
dt = datetime.datetime.fromtimestamp(timestamp / 1000)

# Obtén el año, minutos y segundos del objeto datetime
year = dt.year
minutes = dt.minute
seconds = dt.second

# Obtén el precio del Bitcoin en Binance
symbol = 'BTCUSDT'  # Puedes cambiar el símbolo según tus necesidades
price = client.get_symbol_ticker(symbol=symbol)['price']

# Imprime el resultado
print(f"Precio de Bitcoin (Año: {year}, Minutos: {minutes}, Segundos: {seconds}): {price}")


In [ ]:
pip install yfinance


In [2]:
import yfinance as yf
from datetime import datetime

def obtener_precio_bitcoin():
    # Obtener el objeto ticker de Bitcoin
    ticker = yf.Ticker("BTC-USD")
    
    # Obtener el historial de precios con el rango de tiempo deseado
    historial = ticker.history(period="1s")  # Obtiene el precio más reciente
    
    # Obtener el precio más reciente
    precio_actual = historial["Close"].iloc[-1]
    
    return precio_actual

# Obtener el precio actual del Bitcoin
precio = obtener_precio_bitcoin()

# Imprimir el precio
print("Precio actual de Bitcoin:", precio)


Precio actual de Bitcoin: 26867.669921875


In [ ]:
#Lamentablemente, la API de Yahoo Finance no proporciona datos de precios del Bitcoin con una granularidad temporal de año, minutos y segundos. 
# La API de Yahoo Finance generalmente ofrece datos históricos en intervalos diarios, semanales o mensuales.

In [ ]:
#por lo que probe implementando COINGECKO que trae el valor un poco mas actual

In [2]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from datetime import datetime
import time
import schedule 
def obtener_precio_bitcoin():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd&include_last_updated_at=true"
    
    # Hacer la solicitud a la API de CoinGecko
    response = requests.get(url)
    data = response.json()
    
    # Obtener el precio y la marca de tiempo
    precio = data['bitcoin']['usd']
    marca_tiempo = datetime.fromtimestamp(data['bitcoin']['last_updated_at'])
    
    return precio, marca_tiempo

# Obtener el precio actual del Bitcoin
precio, marca_tiempo = obtener_precio_bitcoin()

while True:
    inicio_tiempo = time.time()

    precio, marca_tiempo = obtener_precio_bitcoin()
    print("Precio actual de Bitcoin:", precio)
    print("Marca de tiempo:", marca_tiempo)

    fin_tiempo = time.time()
    tiempo_transcurrido = fin_tiempo - inicio_tiempo

    tiempo_espera = max(60 - tiempo_transcurrido, 0)
    time.sleep(tiempo_espera)

Precio actual de Bitcoin: 26898
Marca de tiempo: 2023-05-18 21:34:50
Precio actual de Bitcoin: 26898
Marca de tiempo: 2023-05-18 21:34:50
Precio actual de Bitcoin: 26898
Marca de tiempo: 2023-05-18 21:34:50


KeyboardInterrupt: 

In [ ]:
# creamos el codigo para que nos de cada 1 minuto la actualizacion real del bitcoin , podriamos tambien pararlo y que cada X cantidad de tiempo se ejecute solo 
#podriamos automatizar este proceso implementando DAG de apache airflow aumentando la eficiencia 

In [1]:
pip install apache-airflow

In [3]:
import requests
from datetime import datetime
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import datetime as dt
from time import sleep

def obtener_precio_bitcoin():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd&include_last_updated_at=true"

    # Hacer la solicitud a la API de CoinGecko
    response = requests.get(url)
    data = response.json()

    # Obtener el precio y la marca de tiempo
    precio = data['bitcoin']['usd']
    marca_tiempo = datetime.fromtimestamp(data['bitcoin']['last_updated_at'])

    return precio, marca_tiempo

def ejecutar_codigo():
    precio, marca_tiempo = obtener_precio_bitcoin()
    print("Precio actual de Bitcoin:", precio)
    print("Marca de tiempo:", marca_tiempo)


default_args = {
    'owner': 'airflow',
    'start_date': dt(2023, 5, 1),
    'retries': 0
}

dag = DAG('actualizacion_bitcoin', default_args=default_args, schedule_interval='*/1 * * * *')

task = PythonOperator(
    task_id='ejecutar_codigo',
    python_callable=ejecutar_codigo,
    dag=dag
)


ModuleNotFoundError: No module named 'airflow'